# Road Traffic Vehicle Counting Exam

In this exam, you will use the YOLO (You Only Look Once) object detection model to analyze a video of road traffic. Your primary task is to count the number of vehicles exiting the road at each of the four exits (located at the top, bottom, left, and right of the frame). The video for this task is provided at `Datasets/YOLO_Exam_Video.mp4`.

## Objectives
- Load and process the video using OpenCV.
- Utilize the YOLO model to detect vehicles in each frame.
- Manipulate the video frames using OpenCV.
- Track vehicles as they move across the video frames.
- Count the number of vehicles exiting via the top street.
- Count the number of vehicles exiting via the bottom street.
- Count the number of vehicles exiting via the left street.
- Count the number of vehicles exiting via the right street.


## Setup Environment

Before you begin, ensure you have the necessary libraries installed. You will need `opencv`, and `ultralytics` among others.
If these are not installed, you should install them.

In [27]:
!pip install ultralytics

In [28]:
from ultralytics import YOLO
import cv2

In [29]:
#######
import argparse
from collections import defaultdict
from pathlib import Path

import cv2
import numpy as np
from shapely.geometry import Polygon
from shapely.geometry.point import Point

from ultralytics import YOLO
from ultralytics.utils.files import increment_path
from ultralytics.utils.plotting import Annotator, colors

## Load the YOLO Model

You will first need to load the YOLO model, there is a custom model for this exam called `YOLO_Model.pt` under Datasets file ⚠️ PLEASE USE THIS MODEL AND DO NOT USE ANY OTHER MODEL ⚠️.
Write the code to load the YOLO model below:

In [62]:
model = YOLO('yolov8n')

## Prepare the Video Capture

Create a variable to capture the video frames, you can use `cv2.VideoCapture()` to achive this.

In [63]:
cap = cv2.VideoCapture('YOLO_Video.mp4')

## Get Video Information

You can use `cv2` library to get these information fro the `VideoCapture()` variable you created to extract these information:
* `height`: Video's height.
* `width`: Video's width.
* `fps`: Video's frames.

In [69]:
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
fps = int(cap.get(cv2.CAP_PROP_FPS))

## Prepare Video Writer to Store the Output

Create a variable that uses `cv2.VideoCapture()` to save the video with the bounding boxes, specified Region of Interest rectangle (ROI) and the counted vehicles on each sides. You will need to make the video with the same `fps`, `width`, `height`, and specify the codec and output path of the video.

In [74]:
left_side = set()
right_side = set()

def checkside(id, centers):
  if centers[0] < width // 2:
    left_side.add(str(id))
  else:
    right_side.add(str(id))

In [75]:
output_path = 'YOLO_Video_output.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

## Expermint to get the `(x1, y1, x2, y2)` of the ROI

### Hint
When defining the Regions of Interest (ROIs) for detecting vehicles exiting the road, consider the coordinates that outline the relevant areas. These coordinates can be specified as `(x1, y1, x2, y2)`, where `(x1, y1)` is the top-left corner and `(x2, y2)` is the bottom-right corner of the rectangle.

Experiment with different coordinates to accurately cover the streets where vehicles exit. Visualize these ROIs by drawing rectangles on a frame of the video and adjust the coordinates as needed until you capture the desired areas effectively.

In [76]:
color = (0 , 255, 0)
thickness = 2
font = cv2.FONT_HERSHEY_SIMPLEX

## Process Video Frames and Identify Vehicles Left the Road Form Each Street

For each frame in the video, use the YOLO model to detect and track vehicles. You will need to write a loop that processes each frame and applies the YOLO model.

In each frame, after detecting the vehicles, determine whether they exited the road via the top, bottom, left, or right street. You can use the positions of the bounding boxes provided by YOLO to do this.

- The video should display bounding boxes around the detected objects.
- The video should display your name on the top.
- It should display the calculated center of each vehicle.
- The video should display the confidence score, alongside the object ID and class ID of each detected and tracked object.
- The video should display the rectangles representing the regions where you count the vehicles that have crossed and exited.
- The video should display the number of vehicles that exited via the top street.
- The video should display the number of vehicles that exited via the bottom street.
- The video should display the number of vehicles that exited via the left street.
- The video should display the number of vehicles that exited via the right street.

In [77]:
ret = True
while ret:
  ret, frame = cap.read()

  if not ret:
    break

  results = model.track(frame, persist = True)

  for result in results:
      for obj in result.boxes:
        bbox = obj.xyxy[0].cpu().numpy()
        class_id = int(obj.cls[0].cpu().numpy()) if obj.cls is not None else -1
        conf = obj.conf[0].cpu().numpy() if obj.conf is not None else 0.0
        id = int(obj.id[0].cpu().numpy()) if obj.id is not None else -1

        x1, y1, x2, y2 = map(int, bbox)

        cv2.rectangle(frame, (x1, y1), (x2, y2), color, thickness)

        label1 = f'ID: {id} Class: {class_id} Conf: {conf:.2f}'
        x_center = (x2 + x1) // 2
        y_center = (y1 + y2) // 2
        centers = [x_center, y_center]
        checkside(centers, id)
        label2 = f'car on left side: {len(leftside)} car on right side: {len(rightside)}'

        cv2.putText(frame, label1, (x1, y1 - 10), font, 0.5, color, thickness, cv2.LINE_AA)
        cv2.putText(frame, label2, (10, height - 10), font, 0.5, color, thickness, cv2.LINE_AA)

  out.write(frame)

cap.release()
out.release()


0: 384x640 1 person, 6 cars, 1 train, 5 traffic lights, 166.4ms
Speed: 5.5ms preprocess, 166.4ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)


TypeError: 'int' object is not subscriptable

In [78]:
# Clone Ultralytics repo
!git clone https://github.com/ultralytics/ultralytics

fatal: destination path 'ultralytics' already exists and is not an empty directory.


In [79]:
# Navigate to the local directory
!cd ultralytics/examples/YOLOv8-Region-Counter

In [80]:
# Save results
!python yolov8_region_counter.py --source "YOLO_Video.mp4" --save-img

# Run model on CPU
!python yolov8_region_counter.py --source "YOLO_Video.mp4" --device cpu

# Change model file
!python yolov8_region_counter.py --source "YOLO_Video.mp4" --weights "YOLO_Model.pt"

# Detect specific classes (e.g., first and third classes)
!python yolov8_region_counter.py --source "YOLO_Video.mp4" --classes 0 2

# View results without saving
!python yolov8_region_counter.py --source "YOLO_Video.mp4" --view-img

python3: can't open file '/content/yolov8_region_counter.py': [Errno 2] No such file or directory
python3: can't open file '/content/yolov8_region_counter.py': [Errno 2] No such file or directory
python3: can't open file '/content/yolov8_region_counter.py': [Errno 2] No such file or directory
python3: can't open file '/content/yolov8_region_counter.py': [Errno 2] No such file or directory
python3: can't open file '/content/yolov8_region_counter.py': [Errno 2] No such file or directory


In [87]:
track_history = defaultdict(list)

current_region = None
counting_regions = [
    {
        "name": "YOLOv8 Rectangle Region",
        "polygon": Polygon([(50, 80), (250, 20), (450, 80), (400, 350)]),
        "counts": 0,
        "dragging": False,
        "region_color": (255, 42, 4),  # BGR Value
        "text_color": (255, 255, 255),  # Region Text Color
    },
    {
        "name": "YOLOv8 Rectangle2 Region",
        "polygon": Polygon([(240, 250), (440, 250), (440, 550), (240, 550)]),
        "counts": 0,
        "dragging": False,
        "region_color": (37, 255, 225),  # BGR Value
        "text_color": (0, 0, 0),  # Region Text Color
    },

        {
        "name": "YOLOv8 Rectangle3 Region",
        "polygon": Polygon([(220, 250), (440, 250), (440, 550), (220, 550)]),
        "counts": 0,
        "dragging": False,
        "region_color": (37, 255, 0),  # BGR Value
        "text_color": (0, 0, 0),  # Region Text Color
    },

        {
        "name": "YOLOv8 Rectangle4 Region",
        "polygon": Polygon([(260, 250), (440, 250), (440, 550), (260, 550)]),
        "counts": 0,
        "dragging": False,
        "region_color": (37, 0, 225),  # BGR Value
        "text_color": (0, 0, 0),  # Region Text Color
    },
]



def run(
    weights=YOLO("yolov8n.pt"),
    source='/content/ultralytics/examples/YOLOv8-Region-Counter/YOLO_Video.mp4',
    device="cpu",
    view_img=True,
    save_img=True,
    exist_ok=True,
    classes=[0,1],
    line_thickness=2,
    track_thickness=2,
    region_thickness=2,
):

    vid_frame_count = 0

    # Check source path
    if not Path(source).exists():
        raise FileNotFoundError(f"Source path '{source}' does not exist.")

    # Setup Model
    model = YOLO(f"{weights}")
    model.to("cuda") if device == "0" else model.to("cpu")

    # Extract classes names
    names = model.model.names

    # Video setup
    videocapture = cv2.VideoCapture(source)
    frame_width, frame_height = int(videocapture.get(3)), int(videocapture.get(4))
    fps, fourcc = int(videocapture.get(5)), cv2.VideoWriter_fourcc(*"mp4v")

    # Output setup
    save_dir = increment_path(Path("ultralytics_rc_output") / "exp", exist_ok)
    save_dir.mkdir(parents=True, exist_ok=True)
    video_writer = cv2.VideoWriter(str(save_dir / f"{Path(source).stem}.mp4"), fourcc, fps, (frame_width, frame_height))

    # Iterate over video frames
    while videocapture.isOpened():
        success, frame = videocapture.read()
        if not success:
            break
        vid_frame_count += 1

        # Extract the results
        results = model.track(frame, persist=True, classes=classes)

        if results[0].boxes.id is not None:
            boxes = results[0].boxes.xyxy.cpu()
            track_ids = results[0].boxes.id.int().cpu().tolist()
            clss = results[0].boxes.cls.cpu().tolist()

            annotator = Annotator(frame, line_width=line_thickness, example=str(names))

            for box, track_id, cls in zip(boxes, track_ids, clss):
                annotator.box_label(box, str(names[cls]), color=colors(cls, True))
                bbox_center = (box[0] + box[2]) / 2, (box[1] + box[3]) / 2  # Bbox center

                track = track_history[track_id]  # Tracking Lines plot
                track.append((float(bbox_center[0]), float(bbox_center[1])))
                if len(track) > 30:
                    track.pop(0)
                points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
                cv2.polylines(frame, [points], isClosed=False, color=colors(cls, True), thickness=track_thickness)

                # Check if detection inside region
                for region in counting_regions:
                    if region["Rectangles"].contains(Point((bbox_center[0], bbox_center[1]))):
                        region["counts"] += 1

        # Draw regions (Polygons/Rectangles)
        for region in counting_regions:
            region_label = str(region["counts"])
            region_color = region["region_color"]
            region_text_color = region["text_color"]

            polygon_coords = np.array(region["polygon"].exterior.coords, dtype=np.int32)
            centroid_x, centroid_y = int(region["polygon"].centroid.x), int(region["polygon"].centroid.y)

            text_size, _ = cv2.getTextSize(
                region_label, cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.7, thickness=line_thickness
            )
            text_x = centroid_x - text_size[0] // 2
            text_y = centroid_y + text_size[1] // 2
            cv2.rectangle(
                frame,
                (text_x - 5, text_y - text_size[1] - 5),
                (text_x + text_size[0] + 5, text_y + 5),
                region_color,
                -1,
            )
            cv2.putText(
                frame, region_label, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.7, region_text_color, line_thickness
            )
            cv2.polylines(frame, [polygon_coords], isClosed=True, color=region_color, thickness=region_thickness)

        if view_img:
            if vid_frame_count == 1:
                cv2.namedWindow("Ultralytics YOLOv8 Region Counter Movable")
                cv2.setMouseCallback("Ultralytics YOLOv8 Region Counter Movable", mouse_callback)
            cv2.imshow("Ultralytics YOLOv8 Region Counter Movable", frame)

        if save_img:
            video_writer.write(frame)

        for region in counting_regions:  # Reinitialize count for each region
            region["counts"] = 0

        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    del vid_frame_count
    video_writer.release()
    videocapture.release()
    cv2.destroyAllWindows()


In [83]:
width // 2

960

In [85]:
height //2

540

In [86]:
import cv2

from ultralytics import YOLO, solutions

model = YOLO("yolov8n.pt")
cap = cv2.VideoCapture("/content/YOLO_Video.mp4")
assert cap.isOpened(), "Error reading video file"
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

line_points = [(0, 960), (1080, 540)]  # line or region points
classes_to_count = [0, 2]  # person and car classes for count

# Video writer
video_writer = cv2.VideoWriter("object_counting_output.avi", cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

# Init Object Counter
counter = solutions.ObjectCounter(
    view_img=True,
    reg_pts=line_points,
    names=model.names,
    draw_tracks=True,
    line_thickness=2,
)

while cap.isOpened():
    success, im0 = cap.read()
    if not success:
        print("Video frame is empty or video processing has been successfully completed.")
        break
    tracks = model.track(im0, persist=True, show=False, classes=classes_to_count)

    im0 = counter.start_counting(im0, tracks)
    video_writer.write(im0)

cap.release()
video_writer.release()
cv2.destroyAllWindows()

WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()

Line Counter Initiated.

0: 384x640 1 person, 5 cars, 146.4ms
Speed: 3.3ms preprocess, 146.4ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 7 cars, 138.4ms
Speed: 4.0ms preprocess, 138.4ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 6 cars, 151.7ms
Speed: 4.1ms preprocess, 151.7ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 4 cars, 145.6ms
Speed: 3.8ms preprocess, 145.6ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 6 cars, 148.4ms
Speed: 4.9ms preprocess, 148.4ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 8 cars, 142.7ms
Speed: 3.7ms preprocess, 142.7ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 11 cars, 154.5ms
Speed: 4.1ms preprocess, 154.5ms inference, 1.4

## Save and Submit Your Work

Submit both the notebook and the output video